# Description:

* Quick and dirty creation of a 16S rRNA tree of OTUs

In [1]:
workDir = '/home/bryan/ERA/data/MiSeq/20170417_run1/fasttree/'
OTUFileDir = '/home/bryan/ERA/data/MiSeq/20170417_run1/OTU_binning/'
OTUFile = 'otusn.pick.fasta'


nprocs = 20

In [2]:
import os
import numpy as np
from cogent.app.fasttree import build_tree_from_alignment
from cogent import DNA, LoadSeqs
from Bio import Entrez
Entrez.email = "chuck.peperanney@gmail.com"
from IPython.display import display, Image, SVG



In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
library(dplyr)
library(tidyr)
library(ggplot2)

/opt/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/opt/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/opt/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)


In [5]:
if not os.path.isdir(workDir):
    os.mkdir(workDir)

In [6]:
%cd $workDir

/home/bryan/ERA/data/MiSeq/20170417_run1/fasttree


In [7]:
!cd $workDir; ln -f -s $OTUFileDir$OTUFile

In [8]:
!cd $workDir; head $OTUFile

>OTU.4
TACGTAGGGCGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGCTTGTCACGTCGGGTGTGAAAGCCCGGGGCTTAACCCCGGGTCTGCATTCGATACGGGCTAGCTAGAGTGTGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCATTACTGACGCTGAGGAGCGAAAGCGTGGGGAGCGAACAGG
>OTU.5
TACGTAGGGCGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGTCTGTCGCGTCGGATGTGAAAGCCCGGGGCTTAACCCCGGGTCTGCATTCGATACGGGCAGACTAGAGTGTGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCATTACTGACGCTGAGGAGCGAAAGCGTGGGGAGCGAACAGG
>OTU.6
CACGTAGGGGACGAGCGTTGTCCGGATTCATTGGGCGTAAAGCGCGCGTAGGCGGCTCGGAAAGTCGGTCGTGAAATGCCGGGGCTCAACCCCGGGACTGCGTCCGATACTTCCGGGCTGGAGGTAGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCTTACCTGACGCTGAGGCGCGAAAGCTAGGGGAGCGAACAGG
>OTU.7
TACGTAGGGCGCAAGCGTTATCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGTTTGTCGCGTCTGCCGTGAAAGTCCGGGGCTCAACTCCGGATCTGCGGTGGGTACGGGCAGACTAGAGTGATGTAGGGGAGACTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGATGGCGAAGGCAGGTCTCTGGG

In [9]:
!printf "Number of OTUs in fasta: "
!cd $workDir; grep -c ">" $OTUFile

Number of OTUs in fasta: 13919


## Using SSU-Align to align seqs and masking based on alignment posterior probabilities.

In [10]:
!cd $workDir; ssu-prep -f -x -b 50 --rfonly --dna $OTUFile ssu_aln 20 

# ssu-prep :: prepare SSU rRNA sequences for parallel ssu-align jobs
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: ssu-prep -x -f -b 50 --dna --rfonly otusn.pick.fasta ssu_aln 20
# date:    Mon Apr 24 13:21:53 2017
#
# Validating input sequence file ... done.
#
# Preparing 20 ssu-align jobs ...
# Partitioning seqs with goal of equalizing total number of nucleotides per job ...
#
# output file name      description                                           
# --------------------  ------------------------------------------------------
  ssu_aln/otusn.pick.fasta.1  partition  1 FASTA sequence file (696 seqs; 176235 nt)
  ssu_aln/otusn.pick.fasta.2  partition  2 FASTA sequence file (696 seqs; 176270 nt)
  ssu_aln/otusn.pick.fasta.3  partition  3 FASTA sequence file (696 seqs; 176204 nt)
  ssu_aln/otusn.pick.fasta.4

In [11]:
!cd $workDir; ./ssu_aln.ssu-align.sh

# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.1 ssu_aln/ssu_aln.1 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.2 ssu_aln/ssu_aln.2 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.3 ssu_aln/ssu_aln.3 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.4 ssu_aln/ssu_aln.4 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.5 ssu_aln/ssu_aln.5 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.6 ssu_aln/ssu_aln.6 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.7 ssu_aln/ssu_aln.7 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.8 ssu_aln/ssu_aln.8 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.9 ssu_aln/ssu_aln.9 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/otusn.pick.fasta.10 s

In [12]:
!cd $workDir; ssu-mask --dna --afa ssu_aln

# ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: ssu-mask --afa --dna ssu_aln
# date:    Mon Apr 24 13:23:33 2017
#
# Masking alignments based on posterior probabilities...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  ssu_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask      output  mask   1582    248   1334
  ssu_aln.bacteria.mask.pdf  output   pdf   1582    248   1334
  ssu_aln.bacteria.mask.afa  output   aln    248      -      -
#
# All attempts to draw structure diagrams of masks were successful.
#
# List of executed commands saved in

In [13]:
!cd $workDir; ~/seq_tools/seq_tools fasta_unwrap ssu_aln/ssu_aln.bacteria.mask.afa  | head

>OTU.4
TACGTAGGGCGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGCTTGTCACGTCGGGTGTGAAAGCCCGGGGCTTAACCCCGGGTCATTCGATACGGGCTAGCTAGAGTGTGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCAACTGACGCTGAGGAGCGAAAGCGTGGGGAGCGAACAGG
>OTU.5
TACGTAGGGCGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGTCTGTCGCGTCGGATGTGAAAGCCCGGGGCTTAACCCCGGGTCATTCGATACGGGCAGACTAGAGTGTGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCAACTGACGCTGAGGAGCGAAAGCGTGGGGAGCGAACAGG
>OTU.6
-ACGTAGGGGACGAGCGTTGTCCGGATTCATTGGGCGTAAAGCGCGCGTAGGCGGCTCGGAAAGTCGGTCGTGAAATGCCGGGGCTCAACCCCGGGACGTCCGATACTTCCGGGCTGGAGGTAGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCTCCTGACGCTGAGGCGCGAAAGCTAGGGGAGCGAACAGG
>OTU.7
TACGTAGGGCGCAAGCGTTATCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGTTTGTCGCGTCTGCCGTGAAAGTCCGGGGCTCAACTCCGGATCGGTGGGTACGGGCAGACTAGAGTGATGTAGGGGAGACTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGATGGCGAAGGCAGGTCTCTGGGCATACTGACGCTGAGGAG

### Using Sulfolobus as the root (acc. X90478).

In [14]:
sso_acc = "X90478"
sso_fa = Entrez.efetch(db="nucleotide", id=sso_acc, rettype="fasta", retmode="text").readlines()

In [15]:
out = open(os.path.join(workDir, 'X90478.fasta'), 'w')
sso_fa_namestrip = sso_fa[1:]
sso_fa_namestrip.insert(0,">%s\n"%sso_acc)
out.writelines(sso_fa_namestrip)
out.close()

In [16]:
!cd $workDir; head -n 4 X90478.fasta

>X90478
TCCTGCCGGTCCCGACCGCTATCGGGGTGGGGCTAAGCCATGGGAGTCGTACGCTCCCGGGCAAGGGAGC
GTGGCGGACGGCTGAGTAACACGTGGCTAACCTACCCTGAGGAGGGAGATAACCCCGGGAAACTGGGGAT
AATCTCCCATAGGCGAGGAGTCCTGGAACGGTTCCTCGCTGAAAGGCTCATGGGCTATTCCCCGCTCATG


### Align the outgroup to the same cm (use the same mask that was established earlier).

In [17]:
!cd $workDir; ssu-align -f -n bacteria --dna --rfonly X90478.fasta sso_aln

# ssu-align :: align SSU rRNA sequences
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: ssu-align -f -n bacteria --dna --rfonly X90478.fasta sso_aln
# date:    Mon Apr 24 13:23:35 2017
#
# Validating input sequence file ... done.
#
# Stage 1: Determining SSU start/end positions and best-matching models...
#
# output file name          description                                
# ------------------------  -------------------------------------------
  sso_aln.tab               locations/scores of hits defined by HMM(s)
  sso_aln.bacteria.hitlist  list of sequences to align with bacteria CM
  sso_aln.bacteria.fa             1 sequence  to align with bacteria CM
#
# Stage 2: Aligning each sequence to its best-matching model...
#
# output file name          description
# ------------------------  ---------------------

In [18]:
!cd $workDir; ssu-mask -s ssu_aln/ssu_aln.bacteria.mask --dna --afa sso_aln/

# ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: ssu-mask -s ssu_aln/ssu_aln.bacteria.mask --afa --dna sso_aln/
# date:    Mon Apr 24 13:23:36 2017
#
# Masking alignments using pre-existing masks...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  sso_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask       input  mask   1582    248   1334
  sso_aln.bacteria.mask.pdf  output   pdf   1582    248   1334
  sso_aln.bacteria.mask.afa  output   aln    248      -      -
#
# All attempts to draw structure diagrams of masks were successful.
#
# List of 

In [19]:
!cd $workDir; /home/bryan/seq_tools/seq_tools fasta_unwrap sso_aln/sso_aln.bacteria.mask.afa | head

>X90478
TACCAGCCCCGCGAG---TGGTCGGGATTACTGGGCCTAAAGCGCCCGTAGCCGGCCCGACAAGTCACTCCTTAAAGACCCCGGCTCAACCGGGGGAAGGGTGATACTGTCGGGCTAGGGGGCGGGAGAGGCCAGCGGTACTCCCGGAGTAGGGGCGAAATCCTCAGATCTCGGGAGGACCACCAGTGGCGAAAGCGGCTGGCTAGAACGCCCGACGGTGAGGGGCGAAAGCCGGGGCAGCAAAAGGG

In [20]:
!cd $workDir; cat sso_aln/sso_aln.bacteria.mask.afa ssu_aln/ssu_aln.bacteria.mask.afa > aln_for_tree.fasta

In [21]:
!cd $workDir; /home/bryan/seq_tools/seq_tools fasta_unwrap aln_for_tree.fasta | head

>X90478
TACCAGCCCCGCGAG---TGGTCGGGATTACTGGGCCTAAAGCGCCCGTAGCCGGCCCGACAAGTCACTCCTTAAAGACCCCGGCTCAACCGGGGGAAGGGTGATACTGTCGGGCTAGGGGGCGGGAGAGGCCAGCGGTACTCCCGGAGTAGGGGCGAAATCCTCAGATCTCGGGAGGACCACCAGTGGCGAAAGCGGCTGGCTAGAACGCCCGACGGTGAGGGGCGAAAGCCGGGGCAGCAAAAGGG
>OTU.4
TACGTAGGGCGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGCTTGTCACGTCGGGTGTGAAAGCCCGGGGCTTAACCCCGGGTCATTCGATACGGGCTAGCTAGAGTGTGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCAACTGACGCTGAGGAGCGAAAGCGTGGGGAGCGAACAGG
>OTU.5
TACGTAGGGCGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGTCTGTCGCGTCGGATGTGAAAGCCCGGGGCTTAACCCCGGGTCATTCGATACGGGCAGACTAGAGTGTGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCAACTGACGCTGAGGAGCGAAAGCGTGGGGAGCGAACAGG
>OTU.6
-ACGTAGGGGACGAGCGTTGTCCGGATTCATTGGGCGTAAAGCGCGCGTAGGCGGCTCGGAAAGTCGGTCGTGAAATGCCGGGGCTCAACCCCGGGACGTCCGATACTTCCGGGCTGGAGGTAGGTAGGGGAGATCGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGATCTCTGGGCCTCCTGACGCTGAGGC

## Inferring and rooting the tree

In [22]:
aln = LoadSeqs(os.path.join(workDir, 'aln_for_tree.fasta'), moltype=DNA)
t_unroot = build_tree_from_alignment(aln, moltype=DNA)

In [23]:
t_rooted = t_unroot.rootedWithTip(sso_acc)

In [24]:
t_rooted.writeToFile(os.path.join(workDir, 'otusn_pick.tree'))

In [25]:
# ladderize tree
!cd $workDir; tree_edit ladderize -h

Usage: /home/bryan/tree_edit/bin/ladderize.r [options]


Options:
	-t TREE, --tree=TREE
		Tree file

	-f FORMAT, --format=FORMAT
		Tree file format (newick or nexus)

	-o OUTNAME, --outname=OUTNAME
		Output file name. [default: modified input file name]

	-m, --multi
		Multiple trees in file? [FALSE]

	-v, --verbose
		Print extra output

	-h, --help
		Show this help message and exit




In [26]:
# ladderize tree
!cd $workDir; /home/bryan/tree_edit/tree_edit ladderize -t otusn_pick.tree

# --old version--

## viewing tree

In [27]:
!cd $workDir; \
    nw_display -i 'visibility:hidden' -l 'visibility:hidden' -b 'opacity:0' \
    -S -s -r otusn_pick.tree > otusn_pick.svg

/bin/sh: 1: nw_display: not found


In [28]:
treePic = os.path.join(workDir, 'otusn_pick.svg')

SVG(filename=treePic)

ExpatError: no element found: line 1, column 0

In [ ]:
!pwd

## tree summary

In [ ]:
!cd $workDir
!printf "Number of tree tips: "
!nw_labels -I otusn_pick.nwk | wc -l

# Making 'bootstrap' trees

* FastTree doesn't write out each bootstrap tree, just bootstrap values on the final tree
* Need the bootstrap trees for concenTRAIT 
* Making the bootstrap trees 'manually' by bootstrapping the alignment and writting the trees

In [ ]:
alnFile = os.path.join(workDir, 'aln_for_tree.fasta')
bootFile = os.path.splitext(alnFile)[0] + '_boot.nwk'
!/home/bryan/tree_edit/bin/FastTree_bootTrees.py -p 20 $alnFile > $bootFile

In [ ]:
!wc -l $bootFile